# Simulate a transcriptome with TE transcripts using polyester

1. Generate transcriptome 
 - Spliced and unspliced transcripts from GENCODE annotation
 - L1 transcripts from full-length L1HS-L1PA6 annotations in reference genome
 OR
 - L1 consensus sequences from RepBase (ask mike for file)

2. Simulate reads with polyester (see code from `./mikes_old_notebook`)
3. Quantify reads with salmon
 - build index of transcriptome (use same transcriptome from step 1)
 - quantify reads with salmon

4. Compare with original count matrix
 - figure out how to get read counts from salmon (https://salmon.readthedocs.io/en/latest/file_formats.html)

In [21]:
import pandas as pd
from pathlib import Path
from collections import defaultdict
import pyranges as pr
from Bio import SeqIO
from src.make_txome import make_txome
from src.simulate import run_polyester

In [22]:
# set outdir
OUTDIR = Path("../results/20231004_benchmark/")
OUTDIR.mkdir(exist_ok=True, parents=True)

## Generating transcriptome 

### Get L1 transcripts from annotations in reference genome

In [23]:
# read parsed rmsk file
# NOTE: ignore has_promoter column for now, not sure if it is accurate
rmsk = pd.read_csv("../resources/hg38.rmsk.tsv", sep="\t")
rmsk = rmsk[(rmsk.repName == "L1HS") & (rmsk.length > 6000)]
rmsk = rmsk[["genoName", "genoStart", "genoEnd", "strand"]].rename(
    columns={
        "genoName": "Chromosome",
        "genoStart": "Start",
        "genoEnd": "End",
        "strand": "Strand",
    }
)
rmsk = rmsk[rmsk.Chromosome == "chr22"]

# save as bedfile
L1_BED = "../resources/hg38_FL_L1HS.bed"
pr.PyRanges(rmsk).to_bed(L1_BED)

# use bedfile to extract sequences from fasta, save to new fasta
GENOME_FA = "../resources/hg38.fa"
L1_FA = "../resources/hg38_FL_L1HS.fa"
!bedtools getfasta -s -fi {GENOME_FA} -bed {L1_BED} -fo {L1_FA}

In [24]:
### Generating transcriptome
# - use the code from make_txome to make a gtf file with the L1 annotations
TXOME_GTF = "../resources/gencode.v44.primary_assembly.basic.annotation.gtf"
make_txome(OUTDIR / "chr22_txome", GENOME_FA, TXOME_GTF, L1_FA, chromosome="chr22")

[src.make_txome: 10-24 14:02:31] {139748869257024} INFO - Using bedtools from /logg/LOG-G4/mcuoco/projects/bulk_te_bench/.conda/bin/bedtools
INFO:src.make_txome:Using bedtools from /logg/LOG-G4/mcuoco/projects/bulk_te_bench/.conda/bin/bedtools
INFO:src.make_txome:Using bedtools from /logg/LOG-G4/mcuoco/projects/bulk_te_bench/.conda/bin/bedtools
[src.make_txome: 10-24 14:02:54] {139748869257024} INFO - Chromosome chr22 fasta written to /iblm/netapp/data4/mcuoco/tmp/tmp_2x2cflc.fa
INFO:src.make_txome:Chromosome chr22 fasta written to /iblm/netapp/data4/mcuoco/tmp/tmp_2x2cflc.fa
[src.make_txome: 10-24 14:03:37] {139748869257024} INFO - Chromosome chr22 gtf written to /iblm/netapp/data4/mcuoco/tmp/tmppery4_1h.gtf
INFO:src.make_txome:Chromosome chr22 gtf written to /iblm/netapp/data4/mcuoco/tmp/tmppery4_1h.gtf
[src.make_txome: 10-24 14:03:37] {139748869257024} INFO - Saving spliceu txome to ../results/20231004_benchmark/chr22_txome
INFO:src.make_txome:Saving spliceu txome to ../results/2023

In [31]:
# update gene map
t2g_3col = pd.read_csv(OUTDIR / "chr22_txome/txome_t2g_3col.tsv", sep="\t", header=None)
t2g_3col[1] = t2g_3col[1].apply(lambda x: "L1HS" if "chr" in x else x)
t2g_3col.iloc[:, :2].to_csv(
    OUTDIR / "chr22_txome/txome_t2g.tsv", sep="\t", header=None, index=False
)

## Simulate reads with polyester

In [26]:
TXOME_FA = OUTDIR / "chr22_txome/txome.fa"

# make count matrix to simulate from
# ~20x coverage ----> reads per transcript = transcriptlength/readlength * 20
# here all transcripts will have ~equal FPKM
# read length = 100
counts = defaultdict(list)
for tx in SeqIO.parse(TXOME_FA, "fasta"):
    counts["tx_id"].append(tx.id)
    for sample in range(0, 3):
        if "ENS" in tx.id:
            counts[sample].append(20 * len(tx.seq) // 100)
        elif "chr" in tx.id:
            counts[sample].append(sample * len(tx.seq) // 100)
        else:
            counts[sample].append(0)

counts = pd.DataFrame(counts).set_index("tx_id")

In [27]:
run_polyester(TXOME_FA, counts, n_jobs=32, outdir=OUTDIR / "chr22_reads")

[src.simulate: 10-24 14:03:43] {139748869257024} INFO - Simulating reads from 4002 transcripts from 3 samples with polyester
INFO:src.simulate:Simulating reads from 4002 transcripts from 3 samples with polyester
INFO:src.simulate:Simulating reads from 4002 transcripts from 3 samples with polyester
[Parallel(n_jobs=32)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   5 out of  32 | elapsed:  1.1min remaining:  6.1min
[Parallel(n_jobs=32)]: Done   9 out of  32 | elapsed:  1.4min remaining:  3.7min
[Parallel(n_jobs=32)]: Done  13 out of  32 | elapsed:  1.6min remaining:  2.4min
[Parallel(n_jobs=32)]: Done  17 out of  32 | elapsed:  1.9min remaining:  1.7min
[Parallel(n_jobs=32)]: Done  21 out of  32 | elapsed:  2.6min remaining:  1.3min
[Parallel(n_jobs=32)]: Done  25 out of  32 | elapsed:  9.4min remaining:  2.6min
[Parallel(n_jobs=32)]: Done  29 out of  32 | elapsed: 12.5min remaining:  1.3min
[Parallel(n_jobs=32)]: Done  32 out of  32 | e

## Run Salmon

In [28]:
# index the transcriptome
!salmon index -t {OUTDIR}/chr22_txome/txome.fa -i {OUTDIR}/chr22_txome_index -k 31

Version Server Response: Not Found
index ["../results/20231004_benchmark/chr22_txome_index"] did not previously exist  . . . creating it
[2023-10-24 14:21:21.610] [jLog] [warning] The salmon index is being built without any decoy sequences.  It is recommended that decoy sequence (either computed auxiliary decoy sequence or the genome of the organism) be provided during indexing. Further details can be found at https://salmon.readthedocs.io/en/latest/salmon.html#preparing-transcriptome-indices-mapping-based-mode.
[2023-10-24 14:21:21.610] [jLog] [info] building index
out : ../results/20231004_benchmark/chr22_txome_index
[2023-10-24 14:21:21.612] [puff::index::jointLog] [info] Running fixFasta

[Step 1 of 4] : counting k-mers
[2023-10-24 14:21:22.138] [puff::index::jointLog] [warning] Entry with header [ENSG00000054611.14-I] was longer than 400000 nucleotides.  This is probably a chromosome instead of a transcript.
[2023-10-24 14:21:22.446] [puff::index::jointLog] [warning] Entry with he

In [36]:
# quantify the reads with salmon
# -g = File containing mapping of transcripts to genes

r1_reads = sorted((OUTDIR / "chr22_reads").glob("*_1.fasta.gz"))
r2_reads = sorted((OUTDIR / "chr22_reads").glob("*_2.fasta.gz"))

for r1, r2 in zip(r1_reads, r2_reads):
    sample = "_".join(r1.stem.split("_")[0:2])
    !salmon quant -g {OUTDIR}/chr22_txome/txome_t2g.tsv -i {OUTDIR}/chr22_txome_index -l A -1 {r1} -2 {r2} -o {OUTDIR}/chr22_quant/{sample} -p 8

Version Server Response: Not Found
### salmon (selective-alignment-based) v1.10.2
### [ program ] => salmon 
### [ command ] => quant 
### [ geneMap ] => { ../results/20231004_benchmark/chr22_txome/txome_t2g.tsv }
### [ index ] => { ../results/20231004_benchmark/chr22_txome_index }
### [ libType ] => { A }
### [ mates1 ] => { ../results/20231004_benchmark/chr22_reads/sample_01_1.fasta.gz }
### [ mates2 ] => { ../results/20231004_benchmark/chr22_reads/sample_01_2.fasta.gz }
### [ output ] => { ../results/20231004_benchmark/chr22_quant/sample_01 }
### [ threads ] => { 8 }
Logs will be written to ../results/20231004_benchmark/chr22_quant/sample_01/logs
[2023-10-24 14:28:04.195] [jointLog] [info] setting maxHashResizeThreads to 8
[2023-10-24 14:28:04.195] [jointLog] [info] Fragment incompatibility prior below threshold.  Incompatible fragments will be ignored.
[2023-10-24 14:28:04.195] [jointLog] [info] Usage of --validateMappings implies use of minScoreFraction. Since not explicitly speci